In [1]:
import time
import pandas as pd
import os
from selenium import webdriver

In [2]:
# 기사정보를 저장하는 함수
def article_save(query1, query2, query3, ds, de):
    # 기사제목, 링크주소, 좋아요, 화나요를 저장할 딕셔너리
    article_dict = { 'title':[], 'link':[], 'like':[], 'angry':[] }
    
    # article_search()에서 입력한 조건으로 기사를 검색한다.
    # 한 페이지당 큰 제목 기사 10개씩
    for start in range(1,52,10) :
        site = f'https://search.naver.com/search.naver?where=news&query={query1}%26{query2}%26{query3}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds}&de={de}&docid=&nso=so%3Ar%2Cp%3Afrom20190101to20200206%2Ca%3Aall&mynews=0&&start={start}&related=0'
        driver.get(site)
        
        time.sleep(1)
        # 한 페이지당 네이버뉴스 링크가 있는 기사의 태그번호를 끝까지 센다.
        # ul_tag = driver.find_element_by_css_selector('#main_pack > div > ul')
        # li_list = ul_tag.find_elements_by_css_selector('li')
        init = 1
        end = (int(init//10)+1)*15
        
        for i in range(init, end+1, 1) :
            # 기사제목을 가져온다. (sp_nws{i}가 없으면 다음 기사 탐색)
            try :
                li_tag = driver.find_element_by_css_selector(f'#sp_nws{i}')
                a_list = li_tag.find_element_by_css_selector('dl > dt > a')

                # 네이버뉴스 링크가 있으면 
                # 딕셔너리에 기사제목과 링크주소 추가 후
                # 기사를 열어본다.
                news_link = driver.find_element_by_css_selector(f'#sp_nws{i} > dl > dd.txt_inline > a')
                if news_link.text == '네이버뉴스':
                    article_dict['title'].append(a_list.text.strip(''))
                    article_dict['link'].append(news_link.get_attribute('href'))
                    news_link.click()
                
                    # 네이버뉴스가 열린 탭으로 이동
                    last_tab = driver.window_handles[-1]
                    driver.switch_to.window(window_name=last_tab)

                    # 좋아요 수와 화나요 수를 가져온다.
                    like = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
                    angry = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')
                    article_dict['like'].append(int(like.text))
                    article_dict['angry'].append(int(angry.text))

                    # 기사 탭을 닫고 검색결과 화면으로 돌아간다.
                    driver.close()
                    first_tab = driver.window_handles[0]
                    driver.switch_to.window(window_name=first_tab)
                
                # 네이버뉴스 링크 없으면 패스
                else:
                    pass
                
            # sp_nws{i}가 없는 경우 패스 (예외처리)
            except :
                pass

        # 내부for문 종료
        init += 10
    # 외부 for문 종료
    
    # 데이터프레임으로 추가
    df = pd.DataFrame.from_dict(article_dict)
    df.to_csv('naver_article.csv', encoding='utf-8-sig', index=False)
    print('저장완료')
    
        # 한 페이지의 기사를 모두 탐색했으면 다음 페이지로 넘어간다.
        # next_page_btn = driver.find_element_by_css_selector('#main_pack > div > div.paging > a.next')
        # next_page_btn.click()

In [3]:
driver = webdriver.Chrome()

# 네이버뉴스를 검색하는 함수
def article_search():
    # 검색어, 기간을 입력한다.
    # 날짜 형식 : yyyy.mm.dd
    query1 = input('키워드1 : ')
    query2 = input('키워드2 : ')
    query3 = input('키워드3 : ')
    ds = input('시작일 : ')
    de = input('종료일 : ')
    
    # 기사 저장하는 함수 호출
    article_save(query1, query2, query3, ds, de)
        
    # 기사 수집이 끝나면 '작업완료' 메시지 출력 후 프로그램 종료    
    return '작업완료'

In [4]:
article_search()

키워드1 : 강남구
키워드2 : 재개발
키워드3 : 재건축
시작일 : 2018.01.01
종료일 : 2019.01.01
저장완료


'작업완료'